# Networking (Polyglot)

In [ ]:
#!import ../nbs/Testing.dib

In [ ]:
#!import ../nbs/Common.fs
#!import ../nbs/Async.fs
#!import ../nbs/Runtime.fs

In [ ]:
open Common

## testPortOpen

In [ ]:
let inline testPortOpen port = async {
    let! ct = Async.CancellationToken
    use client = new System.Net.Sockets.TcpClient ()
    try
        do! client.ConnectAsync ("127.0.0.1", port, ct) |> Async.awaitValueTaskUnit
        return true
    with ex ->
        trace Verbose (fun () -> $"testPortOpen / ex: {ex |> printException}") getLocals
        return false
}

In [ ]:
//// test

testPortOpen 65536
|> Async.runWithTimeout 60
|> _equal (Some false)

21:42:21 #1 [Verbose] testPortOpen / ex: System.ArgumentOutOfRangeException: Specified argument was out of the range of valid values. (Parameter 'port')
FSharpOption<Boolean>
      Value: False


In [ ]:
let inline testPortOpenTimeout timeout port = async {
    let! result =
        testPortOpen port
        |> Async.runWithTimeoutAsync timeout
    return
        match result with
        | None -> false
        | Some result -> result
}

In [ ]:
//// test

testPortOpenTimeout 60 65535
|> Async.RunSynchronously
|> _equal false

21:42:21 #2 [Debug] runWithTimeoutAsync / timeout: 60
False


## waitForPortAccess

In [ ]:
let inline waitForPortAccess timeout status port =
    let rec loop retry = async {
        let! isPortOpen =
            match timeout with
            | None -> testPortOpen port
            | Some timeout -> testPortOpenTimeout timeout port
        if isPortOpen = status
        then return retry
        else
            if retry % 100 = 0 then
                let getLocals () = $"port: {port} / retry: {retry} / {getLocals ()}"
                trace Warn (fun () -> "waitForPortAccess") getLocals
            do! Async.Sleep 10
            return! loop (retry + 1)
    }
    loop 0

In [ ]:
//// test

let port = 5000

let inline lockPort () = async {
    trace Debug (fun () -> "_1") getLocals
    do! Async.Sleep 5000
    let listener = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port)
    trace Debug (fun () -> "_2") getLocals
    listener.Start ()
    trace Debug (fun () -> "_3") getLocals
    do! Async.Sleep 2000
    trace Debug (fun () -> "_4") getLocals
    listener.Stop ()
    trace Debug (fun () -> "_5") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    let! child = lockPort () |> Async.StartChild
    trace Debug (fun () -> "2") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "3") getLocals
    let! retries1 = waitForPortAccess None true port
    trace Debug (fun () -> "4") getLocals
    let! retries2 = waitForPortAccess None false port
    trace Debug (fun () -> "5") getLocals
    do! child
    trace Debug (fun () -> "6") getLocals
    return retries1, retries2
}
|> Async.runWithTimeout 20000
|> function
    | Some (retries1, retries2) ->
        retries1
        |> _isBetween
            (if Runtime.isWindows () then 2 else 2)
            (if Runtime.isWindows () then 5 else 1500)

        retries2
        |> _isBetween
            (if Runtime.isWindows () then 80 else 80)
            (if Runtime.isWindows () then 150 else 600)

        true
    | _ -> false
|> _equal true

21:42:22 #3 [Debug] 1
21:42:22 #4 [Debug] _1
21:42:22 #5 [Debug] 2
21:42:22 #6 [Debug] 3
21:42:24 #7 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
21:42:24 #8 [Warn] waitForPortAccess / port: 5000 / retry: 0
21:42:26 #9 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
21:42:27 #10 [Debug] _2
21:42:27 #11 [Debug] _3
21:42:27 #12 [Debug] 4
21:42:27 #13 [Warn] waitForPortAccess / port: 5000 / retry: 0
21:42:29 #14 [Warn] waitForPortAccess / port: 5000 / retry: 100
21:42:29 #15 [Debug] _4
21:42:29 #16 [Debug] _5
21:42:31 #17 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
21:42:31 #18 [Debug] 5
21:42:31 #19 [Debug] 6
2
2
2
115
115
115
True


In [ ]:
//// test

let port = 5000

let inline lockPort () = async {
    trace Debug (fun () -> "_1") getLocals
    do! Async.Sleep 500
    let listener = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port)
    trace Debug (fun () -> "_2") getLocals
    listener.Start ()
    trace Debug (fun () -> "_3") getLocals
    do! Async.Sleep 200
    trace Debug (fun () -> "_4") getLocals
    listener.Stop ()
    trace Debug (fun () -> "_5") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    let! child = lockPort () |> Async.StartChild
    trace Debug (fun () -> "2") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "3") getLocals
    let! retries1 = waitForPortAccess (Some 60) true port
    trace Debug (fun () -> "4") getLocals
    let! retries2 = waitForPortAccess (Some 60) false port
    trace Debug (fun () -> "5") getLocals
    do! child
    trace Debug (fun () -> "6") getLocals
    return retries1, retries2
}
|> Async.runWithTimeout 2000
|> function
    | Some (retries1, retries2) ->
        retries1
        |> _isBetween
            (if Runtime.isWindows () then 4 else 2)
            (if Runtime.isWindows () then 15 else 150)

        retries2
        |> _isBetween
            (if Runtime.isWindows () then 5 else 0)
            (if Runtime.isWindows () then 20 else 60)

        true
    | _ -> false
|> _equal true

21:42:31 #20 [Debug] 1
21:42:31 #21 [Debug] 2
21:42:31 #22 [Debug] _1
21:42:31 #23 [Debug] 3
21:42:31 #24 [Debug] runWithTimeoutAsync / timeout: 60
21:42:31 #25 [Warn] waitForPortAccess / port: 5000 / retry: 0
21:42:32 #26 [Debug] runWithTimeoutAsync / timeout: 60
21:42:32 #27 [Debug] runWithTimeoutAsync / timeout: 60
21:42:32 #28 [Debug] runWithTimeoutAsync / timeout: 60
21:42:32 #29 [Debug] runWithTimeoutAsync / timeout: 60
21:42:32 #30 [Debug] runWithTimeoutAsync / timeout: 60
21:42:32 #31 [Debug] _2
21:42:32 #32 [Debug] _3
21:42:32 #33 [Debug] 4
21:42:32 #34 [Warn] waitForPortAccess / port: 5000 / retry: 0
21:42:32 #35 [Debug] _4
21:42:32 #36 [Debug] _5
21:42:32 #37 [Debug] runWithTimeoutAsync / timeout: 60
21:42:32 #38 [Debug] 5
21:42:32 #39 [Debug] 6
6
6
6
12
12
12
True


## getAvailablePort

In [ ]:
let inline getAvailablePort timeout initialPort =
    let rec loop port = async {
        let! isPortOpen =
            match timeout with
            | None -> testPortOpen port
            | Some timeout -> testPortOpenTimeout timeout port
        if not isPortOpen
        then return port
        else return! loop (port + 1)
    }
    loop initialPort

In [ ]:
//// test

let port = 5000

let inline lockPorts () = async {
    trace Debug (fun () -> "_1") getLocals
    let listener1 = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port)
    let listener2 = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port + 1)
    trace Debug (fun () -> "_2") getLocals
    listener1.Start ()
    listener2.Start ()
    trace Debug (fun () -> "_3") getLocals
    do! Async.Sleep 4000
    trace Debug (fun () -> "_4") getLocals
    listener1.Stop ()
    listener2.Stop ()
    trace Debug (fun () -> "_5") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    let! child = lockPorts () |> Async.StartChild
    trace Debug (fun () -> "2") getLocals
    do! Async.Sleep 60
    trace Debug (fun () -> "3") getLocals
    let! availablePort = getAvailablePort None port
    trace Debug (fun () -> "4") getLocals
    let! retries = waitForPortAccess None false port
    trace Debug (fun () -> "5") getLocals
    do! child
    trace Debug (fun () -> "6") getLocals
    return availablePort, retries
}
|> Async.runWithTimeout 15000
|> function
    | Some (availablePort, retries) ->
        availablePort |> _equal (port + 2)

        retries
        |> _isBetween
            (if Runtime.isWindows () then 100 else 100)
            (if Runtime.isWindows () then 150 else 1200)

        true
    | _ -> false
|> _equal true

21:42:33 #40 [Debug] 1
21:42:33 #41 [Debug] 2
21:42:33 #42 [Debug] _1
21:42:33 #43 [Debug] _2
21:42:33 #44 [Debug] _3
21:42:33 #45 [Debug] 3
21:42:35 #46 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
21:42:35 #47 [Debug] 4
21:42:35 #48 [Warn] waitForPortAccess / port: 5000 / retry: 0
21:42:36 #49 [Warn] waitForPortAccess / port: 5000 / retry: 100
21:42:37 #50 [Debug] _4
21:42:37 #51 [Debug] _5
21:42:39 #52 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
21:42:39 #53 [Debug] 5
21:42:39 #54 [Debug] 6
5002
118
118
118
True


In [ ]:
//// test

let port = 5000

let inline lockPorts () = async {
    trace Debug (fun () -> "_1") getLocals
    let listener1 = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port)
    let listener2 = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port + 1)
    trace Debug (fun () -> "_2") getLocals
    listener1.Start ()
    listener2.Start ()
    trace Debug (fun () -> "_3") getLocals
    do! Async.Sleep 400
    trace Debug (fun () -> "_4") getLocals
    listener1.Stop ()
    listener2.Stop ()
    trace Debug (fun () -> "_5") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    let! child = lockPorts () |> Async.StartChild
    trace Debug (fun () -> "2") getLocals
    do! Async.Sleep 20
    trace Debug (fun () -> "3") getLocals
    let! availablePort = getAvailablePort (Some 60) port
    trace Debug (fun () -> "4") getLocals
    let! retries = waitForPortAccess (Some 60) false port
    trace Debug (fun () -> "5") getLocals
    do! child
    trace Debug (fun () -> "6") getLocals
    return availablePort, retries
}
|> Async.runWithTimeout 1500
|> function
    | Some (availablePort, retries) ->
        availablePort |> _equal (port + 2)

        retries
        |> _isBetween
            (if Runtime.isWindows () then 10 else 1)
            (if Runtime.isWindows () then 30 else 120)

        true
    | _ -> false
|> _equal true

21:42:39 #55 [Debug] 1
21:42:39 #56 [Debug] 2
21:42:39 #57 [Debug] _1
21:42:39 #58 [Debug] _2
21:42:39 #59 [Debug] _3
21:42:39 #60 [Debug] 3
21:42:39 #61 [Debug] runWithTimeoutAsync / timeout: 60
21:42:39 #62 [Debug] 4
21:42:39 #63 [Warn] waitForPortAccess / port: 5000 / retry: 0
21:42:39 #64 [Debug] _4
21:42:39 #65 [Debug] _5
21:42:40 #66 [Debug] runWithTimeoutAsync / timeout: 60
21:42:40 #67 [Debug] 5
21:42:40 #68 [Debug] 6
5002
16
16
16
True
